In [1]:
import os
from tensorflow.keras import models, layers, activations, optimizers, utils, losses, initializers, metrics, callbacks
import numpy as np 
import cv2 as cv 
import matplotlib as plt 

In [2]:
epochs = 1000
batch_size = 128
patience = 100
learning_rate = 0.001
model_path = 'checkpoints/model.keras'

In [3]:
exists = os.path.exists(model_path)

model = models.load_model(model_path) \
    if exists \
        else models.Sequential([
            layers.Resizing(25, 25),
            layers.Rescaling(scale=1./127.5, offset=-1),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(256, (3, 3),
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(512,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dropout(0.5),
            layers.Dense(1024,
                activation = 'relu',
                kernel_initializer = initializers.RandomNormal()
            ),
            layers.Dense(62,
                activation = 'sigmoid',
                kernel_initializer = initializers.RandomNormal()
            )
        ])

if exists:
    model.summary()
else:
    model.compile(
        optimizer = optimizers.Adam(
        learning_rate = learning_rate
        ),
        loss = losses.SparseCategoricalCrossentropy(),
        metrics = [ 'accuracy' ]
    )

In [4]:
train = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

test = utils.image_dataset_from_directory(
    "Img",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (128, 128),
    batch_size= batch_size
)

Found 61938 files belonging to 62 classes.
Using 49551 files for training.
Found 61938 files belonging to 62 classes.
Using 12387 files for validation.


In [5]:
model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        )
    ]
)

Epoch 1/1000




388/388 [==============================] - 73s 184ms/step - loss: 2.4006 - accuracy: 0.4097 - val_loss: 0.6748 - val_accuracy: 0.7708
Epoch 2/1000
388/388 [==============================] - 72s 186ms/step - loss: 1.0093 - accuracy: 0.6728 - val_loss: 0.4200 - val_accuracy: 0.8433
Epoch 3/1000
388/388 [==============================] - 78s 201ms/step - loss: 0.6965 - accuracy: 0.7607 - val_loss: 0.3018 - val_accuracy: 0.8842
Epoch 4/1000
388/388 [==============================] - 74s 189ms/step - loss: 0.5645 - accuracy: 0.7994 - val_loss: 0.2715 - val_accuracy: 0.8844
Epoch 5/1000
388/388 [==============================] - 72s 185ms/step - loss: 0.4846 - accuracy: 0.8217 - val_loss: 0.2297 - val_accuracy: 0.9045
Epoch 6/1000
388/388 [==============================] - 72s 185ms/step - loss: 0.4168 - accuracy: 0.8459 - val_loss: 0.2008 - val_accuracy: 0.9171
Epoch 7/1000
388/388 [==============================] - 73s 187ms/step - loss: 0.3943 - accuracy: 0.8512 - val_loss: 0.1918 - val